# Propositional Logic

## Atomic Propositions and Logical Connectives

To create atoms and propositional statements, use the `parse_logical` method. For ease of use, create an alias `PL` and supply a string.

In [1]:
from Propositional.logical import *

PL = parse_logical

# create some propositions.
props = [
    PL("A"),
    PL("A implies B"),
    PL("(A or B) and (not (A and B))"),
    PL("A iff B")
]

print(props, sep="\n")


[A, (A → B), ((A ∨ B) ∧ ¬(A ∧ B)), (A ↔ B)]


Alternatively, we can start with a set of atoms and combine them after the fact.

In [2]:
from Propositional.logical import *

PL = parse_logical

# create three atoms: a, b, and c.
a, b, c = [Atom(chr(ord("A")+i)) for i in range(3)]

props = [
    a,
    a >> b,
    (a | b) & (~(a & b)),
    a ^ b
]

print(props, sep="\n")


[A, (A → B), ((A ∨ B) ∧ ¬(A ∧ B)), (A ↔ B)]


## Evaluating logical propositions

To associate truth values to atoms in propositions that were pre-built using `parse_logical`, use the `set_atom_truth_values` method on each connective. To evaluate the truth value of the proposition, use the `bool` built-in function.

In [3]:
from Propositional.logical import *

PL = parse_logical

atoms = []

for prop in props:
    atoms += prop.atoms_contained()

atoms = sorted(list(set(atoms)))

num_atoms = len(atoms)

for i in range(2 ** num_atoms):
    truth_values = [
        i & 1 << a != 0 for a in range(num_atoms)
    ]
    
    atom_truths = ";\t".join([
        " " + str(atoms[i]) if truth_values[i] else str(~atoms[i])
        for i in range(num_atoms)
    ] + [""])
    
    context = {
        atom: truth_value for atom, truth_value in zip(atoms, truth_values)
    }
    
    for prop in props:
        # set proposition truth values.
        prop.set_atom_truth_values(context)
            
    prop_truths =  ";\t\t".join([
        str(prop) + " -> " + str(bool(prop))[0]
        for prop in props
    ])
    
    
    print(atom_truths, prop_truths, sep="\t|\t")
    

    

¬A;	¬B;		|	A -> F;		(A → B) -> T;		((A ∨ B) ∧ ¬(A ∧ B)) -> F;		(A ↔ B) -> T
 A;	¬B;		|	A -> T;		(A → B) -> F;		((A ∨ B) ∧ ¬(A ∧ B)) -> T;		(A ↔ B) -> F
¬A;	 B;		|	A -> F;		(A → B) -> T;		((A ∨ B) ∧ ¬(A ∧ B)) -> T;		(A ↔ B) -> F
 A;	 B;		|	A -> T;		(A → B) -> T;		((A ∨ B) ∧ ¬(A ∧ B)) -> F;		(A ↔ B) -> T


## Basic Arguments

There are a number of basic arguments that need no derivation. For example, there is an argument called 'Modus Ponens' which encapsulates the idea of "if p then q; p; therefore q". 

To use this argument, we do the following.




In [4]:
from Propositional.arguments import *

PL = parse_logical

p = PL("A implies B")
q = PL("A")

if_p = p >> q

print("p;          ", p)
print("q;          ", q)
print("If p then q;", if_p)

mp = ModusPonens(if_p, p)

print("Modus Ponens", "is" if (mp.get_application() is not None) else "isn't", "applicable")

application = mp.apply()

print("Modus Ponens gives", application)
print("Q is application:", application == q)



p;           (A → B)
q;           A
If p then q; ((A → B) → A)
Modus Ponens is applicable
Modus Ponens gives A
Q is application: True


In [5]:
from Propositional.arguments import *

PL = parse_logical

p = PL("A implies B")
q = PL("C or D")

if_p = p >> q
not_q = ~ q

print("p;          ", p)
print("not q;      ", not_q)
print("If p then q;", if_p)

mt = ModusTollens(if_p, not_q)

print("Modus Tollens", "is" if (mp.get_application() is not None) else "isn't", "applicable")

application = mt.apply()

print("Modus Tollens gives", application)
print("not P is application:", application == ~p)




p;           (A → B)
not q;       ¬(C ∨ D)
If p then q; ((A → B) → (C ∨ D))
Modus Tollens is applicable
Modus Tollens gives ¬(A → B)
not P is application: True


## Derivations

### Direct Derivations

By using a set of propositions as axioms, we can create derivations. For example, if we specify that `A` is true and `C` is false, then we can prove that if `A`, then; if `B` then `C`, then `B` must be false. 


In [6]:
from Propositional.derivations import *
PL = parse_logical

axioms = [
    PL("A"), PL("not C"), PL("A implies (B implies C)")
]
consequence = PL("not B")

derivation = [
    ModusPonens(PL("A"), PL("A implies (B implies C)")),
    ModusTollens(PL("not C"), PL("B implies C"))
]

dd = DirectDerivation(axioms, consequence, derivation)

print(dd.verify())


(A → (B → C)). A. ¬C ⊢ ¬B
Show ¬B:
| 1  (B → C)               mp pr2, pr1
| 2  ¬B                    mt pr3, 1
| 3                        dd 2
Q.E.D.


Derivations can also be conditional or indirect.

### Conditional Derivations

In a conditional derivation, the consequence is in the form `A → B`, and so instead of proving `P1, P2, ..., Pk ⊢ A → B`, we can prove `P1, P2, ..., Pk, A ⊢ B`, which is functionally the exact same derivation. When creating a conditional derivation, specify the axioms and consequence as usual for a direct derivation, but in the derivation list, assume that `A` is now part of the axioms, and the derivation should end with a line deriving `B`.



In [7]:
from Propositional.derivations import *
PL = parse_logical

axioms = [
    PL("(not A) or B")
]
consequence = PL("A implies B")

derivation = [
    ModusTollendoPonens(PL("(not A) or B"), PL("A"))
]

cd = ConditionalDerivation(axioms, consequence, derivation)

print(cd.verify())



(¬A ∨ B) ⊢ (A → B)
Show (A → B):
| 1  A                     asm
| 2  B                     mtp pr1, 1
| 3                        cd 2
Q.E.D.



### Indirect Derivations

In an indirect derivation, we aim to show that the premises being true while the consequence is false leads to a contradiction. So instead of proving `P1, P2, ..., Pk ⊢ C`, we can prove `P1, P2, ..., Pk, ¬C ⊢ ⊥`, which is functionally the exact same derivation. When creating an indirect derivation, again, set up axioms and consequence as usual, but in the derivation, assume `¬C` is now part of the axioms, and the derivation should end with a contradiction object.



In [8]:
from Propositional.derivations import *
PL = parse_logical

axioms = [
    PL("A or B")
]

consequence = PL("not ((not A) and (not B))")

derivation = [
    DoubleNegation(PL("not (not ((not A) and (not B)))"), PL("(not A) and (not B)")),
    Simplification(PL("(not A) and (not B)"), PL("not A")),
    Simplification(PL("(not A) and (not B)"), PL("not B")),
    ModusTollendoPonens(PL("A or B"), PL("not A")),
    Contradiction(PL("B"), PL("not B"))
]

id_ = IndirectDerivation(axioms, consequence, derivation)

print(id_.verify())




(A ∨ B) ⊢ ¬(¬A ∧ ¬B)
Show ¬(¬A ∧ ¬B):
| 1  ¬¬(¬A ∧ ¬B)           asm
| 2  (¬A ∧ ¬B)             dn 1
| 3  ¬A                    s 2
| 4  ¬B                    s 2
| 5  B                     mtp pr1, 3
| 6  ⊥                     id 5, 4
Q.E.D.


As you might recognize, the conditional derivation was part of the more complex derivation, one for the derivation of `((¬A ∨ B) ↔ (A → B))`, the material equivalence of the disjunction of the negation of the antecedent, and of the consequence, with the implication. The indirect derivation above was again similarly part of a more complex derivation, that of `((A ∨ B) ↔ ¬(¬A ∧ ¬B))`, one version of De Morgan's Laws.

### Subderivations

As noted, it is oftentimes necessary to nest derivations within each other. For example to prove `A ↔ B`, one must independently prove `A → B` and `B → A`. Therefore instead of basic arguments, we can use 'derived arguments', in the form of subderivations. We can create these from normal derivations of any variety, and can be used in larger derivations as long as the subderivation's axioms / assumptions have all been previously derived. In this case, as long as the subderivation is valid, the consequence of the subderivation is introduced as a new derivation. 



In [9]:
from Propositional.theorem import *
PL = parse_logical

# initialize the derivation. The overall form is of P -> Q, with P = A implies B and Q = (not A) or B

axioms = []
consequences = PL("(A implies B) implies ((not A) or B)")
derivation = []

dd = ConditionalDerivation(
    axioms=[],
    consequence=PL("(A implies B) implies ((not A) or B)"),
    derivation=[
        # Assume A -> B
        # Show ((not A) or B)
        IndirectDerivation(
            # Assume ~((not A) or B)
            axioms=[], 
            consequence=PL("((not A) or B)"), 
            derivation=[
                ConditionalDerivation(
                    # Show A -> ((not A) or B)
                    # Assume A
                    axioms=[], 
                    consequence=PL("A implies ((not A) or B)"), 
                    derivation=[
                        ModusPonens(PL("A implies B"), PL("A")),
                        Addition(PL("B"), PL("((not A) or B)"))
                    ]
                ),
                ConditionalDerivation(
                    # Show ~A -> ((not A) or B)
                    # Assume ~A
                    axioms=[], 
                    consequence=PL("(not A) implies ((not A) or B)"), 
                    derivation=[
                        Addition(PL("not A"), PL("((not A) or B)"))
                    ]
                ),
                TertiumNonDatur(therefore=PL("A or (not A)")),
                DisjunctionElimination(PL("A or (not A)"), PL("A implies ((not A) or B)"), PL("(not A) implies ((not A) or B)"), therefore=PL("(not A) or B"))
            ]
        )
    ]
)

# verify the proof.
print(dd.verify())


 ⊢ ((A → B) → (¬A ∨ B))
Show ((A → B) → (¬A ∨ B)):
| 1  (A → B)               asm
| 2  Show (¬A ∨ B):        
|    | 3  ¬(¬A ∨ B)        asm
|    | 4  Show (A → (¬A ∨ B)):
|    |    | 5  A           asm
|    |    | 6  B           mp 1, 5
|    |    | 7  (¬A ∨ B)    a 6
|    |    | 8              cd 7
|    | 9  Show (¬A → (¬A ∨ B)):
|    |    | 10 ¬A          asm
|    |    | 11 (¬A ∨ B)    a 10
|    |    | 12             cd 11
|    | 13 (A ∨ ¬A)         TND
|    | 14 (¬A ∨ B)         DE 13, 4, 9
|    | 15                  id 14
| 16                       cd 2
Q.E.D.


## Basic Arguments

There are a set of natural deductions, such as Modus Ponens, Modus Tollens, etc., but also a set of basic arguments which are commonly known but are quite useful to cut down on the length and complexity of longer proofs. Therefore we can define basic arguments as a valid derivation paired with an argument name.

## Basic Search

Currently there is functionality to solve certain simple derivations that require no contradictions or subderivations. As this project progresses, more and more derivations will be solvable through this tool.


In [10]:
from Propositional.search import *
PL = parse_logical

axiom_sets = [
    [
        PL("A"), PL("not C"), PL("(A implies (B implies C))")
    ],
    [
        PL("A implies B"), PL("B implies C")
    ]

]
consequence_set = [
    PL("not B"),
    PL("A implies C")
]

derivation_sets = [
    [
        ModusPonens(PL("A"), PL("(A implies (B implies C))")),
        ModusTollens(PL("B implies C"), PL("not C"))
    ],
    [
        ModusPonens(PL("A"), PL("A implies B")),
        ModusPonens(PL("B"), PL("B implies C"))
    ]
]

for axiom, consequence, derivation in zip(axiom_sets, consequence_set, derivation_sets):
    derivation_type = DirectDerivation
    if type(consequence) is LOGICAL_CONNECTIVES['implies']:
        derivation_type = ConditionalDerivation

    p1 = derivation_type(
        axiom,
        consequence,
        derivation
    )

    print("Handwritten Proof")
    print("---------")

    print("\n  ".join([""] + (p1verification := p1.verify()).split("\n"))[1:])
    print("---------")

    bfs = BFSDirectDerivation(
        axiom,
        consequence
    )

    derivation2 = bfs.search()

    p2 = derivation_type(
        axiom,
        consequence,
        derivation2
    )

    print("---------")
    print("Computer Generated Proof")
    print("---------")

    print("\n  ".join([""] + (p2verification := p2.verify()).split("\n"))[1:])
    print("---------")

    # print(f"Proof is identical: {p1verification == p2verification}")
    print("---------")




Handwritten Proof
---------
  (A → (B → C)). A. ¬C ⊢ ¬B
  Show ¬B:
  | 1  (B → C)               mp pr2, pr1
  | 2  ¬B                    mt 1, pr3
  | 3                        dd 2
  Q.E.D.
---------
Component not in environment (B → C)
---------
Computer Generated Proof
---------
  (A → (B → C)). A. ¬C ⊢ ¬B
  Show ¬B:
  | 1  (B → C)               mp pr2, pr1
  | 2  ¬B                    mt pr3, 1
  | 3                        dd 2
  Q.E.D.
---------
---------
Handwritten Proof
---------
  (A → B). (B → C) ⊢ (A → C)
  Show (A → C):
  | 1  A                     asm
  | 2  B                     mp 1, pr1
  | 3  C                     mp 2, pr2
  | 4                        cd 3
  Q.E.D.
---------
Component not in environment B
---------
Computer Generated Proof
---------
  (A → B). (B → C) ⊢ (A → C)
  Show (A → C):
  | 1  A                     asm
  | 2  B                     mp 1, pr1
  | 3  C                     mp pr2, 2
  | 4                        cd 3
  Q.E.D.
---------
---------


In [ ]:
## Proposition Simplification

It may turn out that a particular result, but through various techniques we can simplify logical propositions.

### Karnaugh Simplification

Using the idea of min-terms and max-terms, and using the sum-of-min-terms and product-of-max-terms, we can start with a basic way of breaking down the logical proposition. Then, (Not implemented yet) we can simplify each term as it progresses through to create a much more simplified statement, all in terms of conjunctions and disjunctions.

### Breadth First Search

Using breath first search with pruning using a priority queue, we can simplify much further than Karnaugh maps for many cases. During breadth first search, at each expanded node, we check if the expanded proposition is logically equivalent to the proposition we're simplifying. If so, we return it. Otherwise, we create conjunctions, disjunctions, material implications, material equivalence, and negations of the expanded node with any of the previously expanded nodes, and provided the length of the string representation is not larger than the length of the string representation of the proposition we start with, we add it to the list of propositions to be expanded upon. 


In [11]:
from Propositional.simplify import *

PL = parse_logical

""" 
Try running this on your local machine! See how long it takes your computer to run this code!
"""

props = [
    PL("((not A) or B)"),
    PL("(P implies (Q implies R))"),
    PL("((X implies Y) implies (Y implies (Z or W)))"),
]

for prop in props:
    print("-"*60)
    print("-"*60)

    print(f"Proposition to simplify: {prop}")

    # v = True, False, or 'progress bar'
    v = 'progress bar'
    other = True

    print("-"*50)

    bfs = BFS(prop)
    bfs.simplify(verbose=v)
    diagnostics = bfs.diagnostics
    solution_no_filter = diagnostics.simplification
    print(diagnostics)



------------------------------------------------------------
------------------------------------------------------------
Proposition to simplify: (¬A ∨ B)
--------------------------------------------------

BFS |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0%  / considered = 7 (T=0.0051772594)

+----------------
| Diagnostics
+----------------
| Starting proposition: (¬A ∨ B)
| Ending proposition: (A → B)
| Considered 7 propositions.
| Pruned 72.
| Took 0.005575 seconds.
+----------------
------------------------------------------------------------
------------------------------------------------------------
Proposition to simplify: (P → (Q → R))
--------------------------------------------------

BFS |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0%  / considered = 156 (T=3.9977479)

+----------------
| Diagnostics
+----------------
| Starting proposition: (P → (Q 